## Setup:

In [16]:
import pandas as pd

from tensorflow import keras
from keras.layers import Dense
from keras.models import Sequential

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

## Data Loading and Initial Exploration:

In [17]:
# Load the dataset
file_path = 'Auto Claims.csv'
auto_claims_data = pd.read_csv(file_path)

In [18]:
# Display basic information about the dataset
auto_claims_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Claim ID          100 non-null    int64 
 1   Date of Accident  100 non-null    object
 2   Date of Claim     100 non-null    object
 3   Policy Holder ID  100 non-null    int64 
 4   Vehicle Make      100 non-null    object
 5   Vehicle Model     100 non-null    object
 6   Vehicle Year      100 non-null    int64 
 7   Claim Amount      100 non-null    int64 
 8   Claim Status      100 non-null    object
dtypes: int64(4), object(5)
memory usage: 7.2+ KB


In [19]:
# Display the dataset
auto_claims_data.head()

,Claim ID,Date of Accident,Date of Claim,Policy Holder ID,Vehicle Make,Vehicle Model,Vehicle Year,Claim Amount,Claim Status
0,1,2022-05-10,2022-05-15,1001,Toyota,Camry,2018,5000,Approved
1,2,2022-06-05,2022-06-10,1002,Honda,Accord,2019,3000,Approved
2,3,2022-07-15,2022-07-20,1003,Hyundai,Sonata,2017,8000,Rejected
3,4,2022-08-20,2022-08-25,1004,Ford,Escape,2020,4000,Approved
4,5,2022-09-12,2022-09-15,1005,Toyota,Rav4,2016,2000,Approved


## Data Preprocessing

In [20]:
# Convert categorical variables using label encoding
label_encoder = LabelEncoder()
auto_claims_data['Vehicle Make'] = label_encoder.fit_transform(auto_claims_data['Vehicle Make'])
auto_claims_data['Vehicle Model'] = label_encoder.fit_transform(auto_claims_data['Vehicle Model'])

In [21]:
# Convert dates to ordinal numbers
auto_claims_data['Date of Accident'] = pd.to_datetime(auto_claims_data['Date of Accident']).apply(lambda x: x.toordinal())
auto_claims_data['Date of Claim'] = pd.to_datetime(auto_claims_data['Date of Claim']).apply(lambda x: x.toordinal())

## Feature Selection and Train-Test Split

In [22]:
# Assume 'Claim Status' is the target variable and it is binary
y = auto_claims_data['Claim Status'].apply(lambda x: 1 if x == 'Approved' else 0)
X = auto_claims_data.drop('Claim Status', axis=1)

In [23]:
# Scale the feature data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [24]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

## Model Training

In [25]:
# Define a simple neural network model for binary classification
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [26]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [27]:
# Train the model
history = model.fit(X_train, y_train, epochs=10, validation_split=0.1)

Epoch 1/10
3/3 [==============================] - 1s 223ms/step - loss: 0.6759 - accuracy: 0.6667 - val_loss: 0.6929 - val_accuracy: 0.6250
Epoch 2/10
3/3 [==============================] - 0s 26ms/step - loss: 0.6627 - accuracy: 0.6806 - val_loss: 0.6993 - val_accuracy: 0.6250
Epoch 3/10
3/3 [==============================] - 0s 19ms/step - loss: 0.6518 - accuracy: 0.6528 - val_loss: 0.7042 - val_accuracy: 0.6250
Epoch 4/10
3/3 [==============================] - 0s 17ms/step - loss: 0.6424 - accuracy: 0.6528 - val_loss: 0.7041 - val_accuracy: 0.6250
Epoch 5/10
3/3 [==============================] - 0s 18ms/step - loss: 0.6358 - accuracy: 0.6528 - val_loss: 0.7020 - val_accuracy: 0.6250
Epoch 6/10
3/3 [==============================] - 0s 18ms/step - loss: 0.6307 - accuracy: 0.6528 - val_loss: 0.6957 - val_accuracy: 0.6250
Epoch 7/10
3/3 [==============================] - 0s 30ms/step - loss: 0.6274 - accuracy: 0.6528 - val_loss: 0.6929 - val_accuracy: 0.6250
Epoch 8/10
3/3 [==========

## Model Evaluation

In [28]:
# Using the model's .predict() method to get predicted probabilities
y_probs = model.predict(X_test)

# Apply a threshold to the probabilities to get binary class predictions
# This is a common practice for binary classification problems
y_pred = (y_probs > 0.5).astype(int)

1/1 [==============================] - 0s 71ms/step


In [29]:
# Calculate the metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Print the metrics
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

Accuracy: 0.8000
Precision: 0.8000
Recall: 1.0000
F1 Score: 0.8889


The metrics provided from our model evaluation on the Auto Claims  dataset suggest the following about its performance:

- **Accuracy (0.8000 or 80%):** This indicates that **80%** of the model's predictions are correct. It's a measure of how often the model is correct overall, but it doesn't give us specific insights into how different types of errors are distributed.

- **Precision (0.8000 or 80%):** This suggests that when our model predicts a case as positive (Claim Approved), it is correct **80%** of the time. This is an important metric when the cost of false positives is high.

- **Recall (1.0000 or 100%):** The model has a perfect recall score, meaning it correctly identified all actual positive cases (all Approved claims). This is crucial when the cost of missing a positive case (false negatives) is high.

- **F1 Score (0.8889 or 88.89%):** The F1 score is the harmonic mean of precision and recall, balancing the two metrics. An F1 score of **88.89%** is very high, suggesting that the model has a good balance between precision and recall. This is particularly valuable in cases where both false positives and false negatives are costly.

## Interpretation and Considerations:

- The high **recall** indicates that the model is excellent at identifying positive cases, but the **precision** score suggests there's still a **20%** chance of a positive prediction being incorrect.

- The **accuracy** and **F1** score are also **high**, which overall suggests the model is performing well on this particular dataset.

- However, it's important to consider the context and the dataset's balance. For instance, if the dataset is imbalanced (more Rejected claims than Approved), these metrics might not fully represent the model's performance, as models tend to perform well in the majority class.

- Additionally, we should consider other factors like the model's complexity, the dataset's size and diversity, and the potential impact of different types of errors (false positives vs. false negatives) in the specific use case.

In summary, our model seems to be performing quite well according to these metrics, especially in correctly identifying all positive cases. However, we should always use additional evaluation techniques (like **confusion matrix** analysis or **ROC curves**) for a more comprehensive assessment.